In [0]:
%sql
CREATE SCHEMA prem_catalog.gold


In [0]:
df = spark.read.format('delta')\
    .load('abfss://silver@databricksetestg.dfs.core.windows.net/customers')
display(df)

### ***surrogate key***
### To add incremental number for primary key in the table

In [0]:
from pyspark.sql.functions import monotonically_increasing_id, lit
df = df.withColumn("Dimcustomerkey", monotonically_increasing_id() + lit(1))
df = df.dropDuplicates(subset = ['customer_id'])
df.display()

### ***Dividing old vs new records***

In [0]:
(dbutils.widgets.text('init_load_flag',''))
init_load_flag = int(dbutils.widgets.get('init_load_flag'))

In [0]:
if init_load_flag == 1:
    df_old = spark.sql("select Dimcustomerkey, customer_id, update_date,create_date from prem_catalog.gold.Dimcustomers")

else:
    df_old = spark.sql("select 0 Dimcustomerkey, 0 customer_id, 0 update_date, 0 create_date from prem_catalog.silver.customers_silver where 1=0")

In [0]:
display(df_old)

In [0]:
display(df_new)

In [0]:
df_old = df_old.withColumnRenamed("Dimcustomerkey","old_Dimcustomerkey")
df_old = df_old.withColumnRenamed("customer_id","old_customer_id")  
df_old = df_old.withColumnRenamed("update_date","old_update_date")  
df_old = df_old.withColumnRenamed("create_date","old_create_date")
from pyspark.sql.functions import col
df_join = df.join(df_old, col('customer_id') == col('old_customer_id'), 'left')

display(df_join)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
df_new = df_join.filter(col('old_Dimcustomerkey').isNull())
df_old = df_join.filter(col('old_Dimcustomerkey').isNotNull())
display(df_new)
display(df_old)


In [0]:
display(df_old)

In [0]:
df_old = df_old.drop("old_Dimcustomerkey","old_customer_id","old_update_date")
df_old = df_old.withColumnRenamed("old_create_date","create_date")
df_old = df_old.withColumn("create_date",current_timestamp())
df_old = df_old.withColumn("update_date",current_timestamp())
display(df_old)
display(df_old)

In [0]:
df_new = df_new.drop("old_Dimcustomerkey","old_customer_id","old_update_date","old_create_date")
df_new = df_new.withColumn("create_date",current_timestamp())
df_new = df_new.withColumn("update_date",current_timestamp())
display(df_new)


In [0]:
if init_load_flag == 1:
    max_surrogate = 0

else:
    df_maxsur = spark.sql("select max(Dimcustomerkey) as max_surrogate from prem_catalog.gold.Dimcustomers")
    ###converting df_maxsur to max_surrogate variable
    max_surrogate = df_maxsur.collect()[0]['max_surrogate']
##df_new = df_new.withColumn("Dimcustomerkey", monotonically_increasing_id() + lit(max_surrogate + 1))
df_new = df_new.withColumn("Dimcustomerkey",lit(max_surrogate) + col("Dimcustomerkey"))
df_new = df_new.dropDuplicates(subset = ['customer_id'])

In [0]:
display(df_new)

In [0]:
df_old = df_old.dropDuplicates(subset = ['customer_id'])
display(df_old)

In [0]:
df_final = df_new.unionByName(df_old)
display(df_final)


In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("prem_catalog.gold.Dimcustomers"):
    dlt_obj = DeltaTable.forName(spark,"prem_catalog.gold.Dimcustomers")
    dlt_obj.alias("t").merge(df_final.alias("s"),"t.Dimcustomerkey = s.Dimcustomerkey").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
    df_final.write.mode("overwrite").format('delta').option('path',"abfss://gold@databricksetestg.dfs.core.windows.net/Dimcustomers").saveAsTable("prem_catalog.gold.Dimcustomers")

In [0]:
%sql
DROP TABLE prem_catalog.gold.dimcustomers;